### Setup libraries

In [3]:

import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
matplotlib.use('Qt5Agg')
plt.ion()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.io import loadmat
import glob
from scipy import signal
from scipy.fftpack import fft, ifft
from scipy.signal import butter, lfilter
import mne

### Setup data information

In [4]:
emotion_label = ['neutral', 'sad', 'fear', 'happy']
session1_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]
session2_label = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]
session3_label = [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]
# location = "../SEED_IV/eeg_raw_data"


### Import files

In [5]:
files = glob.glob("../SEED_IV/eeg_raw_data/1/1_20160518.mat")
# files = glob.glob(location+"/*/*.mat")
files

[]

In [6]:
subjects = [loadmat(files[0])]
# for file in files:


IndexError: list index out of range

### Create MNE object

In [ ]:
target_channels = [14, 22, 23, 31, 32, 40]
channel_mapping = ['FT7','FT8','T7','T8','TP7','TP8']
ch_types = ['eeg'] * 6
sampling_freq = 1000

info = mne.create_info(channel_mapping, ch_types=ch_types, sfreq=sampling_freq)
info.set_montage('standard_1020')



In [ ]:
data = np.array(subjects)
trial_no = 0
session1_label[trial_no]
trial = data[trial_no]['cz_eeg1']
channel = trial[target_channels[0]]
channel2 = trial[target_channels[1]]
channel3 = trial[target_channels[2]]
channel4 = trial[target_channels[3]]
channel5 = trial[target_channels[4]]
channel6 = trial[target_channels[5]]

raw_data=np.array([channel,channel2,channel3,channel4,channel5,channel6])
raw_data

In [ ]:
info = mne.create_info(ch_names=channel_mapping,
                       ch_types=ch_types,
                       sfreq=sampling_freq)
simulated_raw = mne.io.RawArray(raw_data, info)
picks = mne.pick_types(simulated_raw.info, meg=False, eeg=True, eog=False,
                       stim=False)

In [ ]:
FFTed_signal = simulated_raw.filter(1, 70., fir_design='firwin')
FFTed_signal.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False);

In [ ]:
resampled_signal = FFTed_signal.resample(150, npad="auto")  # set sampling frequency to 100Hz
resampled_signal.plot_psd(area_mode='range', tmax=10.0, picks=picks);

In [ ]:
data = np.array(subjects)
trial = data[0]['cz_eeg1']
channel = trial[target_channels[0]]
plt.plot(channel)

In [ ]:
time = np.linspace(0,channel.size/1000,channal.size)
plt.plot(time, channel)

### Apply bandpass filter

In [ ]:
taps = signal.firwin(400, [0.1, 0.75], pass_zero=False)
filtered_chanel = signal.convolve(channel, taps, mode='same')
# t = np.arange(0.0, channal.size/sampling_freq, 1/sampling_freq)
t = np.linspace(0,channel.size/1000,channel.size)
plt.figure(figsize=(18,4))
plt.plot(t, channel, alpha=0.4)
plt.plot(t, filtered_chanel)
plt.show()

### Apply FFT

In [ ]:
FFTed_channel = np.fft.fft(filtered_chanel)
plt.plot(FFTed_channel)

### Take the real part of it with the abs command.

In [ ]:
N = round(len(FFTed_channel)/2+1)
FFTed_channel[N-4:N+3]
plt.plot(np.abs(FFTed_channel))

### Amplitude Spectrum

In [ ]:
plt.plot(np.abs(FFTed_channel[:N]))

### Real Physical Values for the Amplitude and Frequency Axes of the FFT

x-Axis: The Frequency Axis of the FFT

In [ ]:
dt = t[1] - t[0]
fa = 1.0/dt # scan frequency
print('dt=%.5fs (Sample Time)' % dt)
print('fa=%.2fHz (Frequency)' % fa)

This frequency is half of the maximum sampling frequency (fa) and is called the Nyquist-Frequency

In [ ]:
X = np.linspace(0, fa/2, N, endpoint=True)
X[:]

In [ ]:
plt.plot(X, np.abs(FFTed_channel[:N]))
plt.xlabel('Frequency ($Hz$)')

y-Axis: The Amplitude of the FFT Signal

In [ ]:
plt.plot(X, 2.0*np.abs(FFTed_channel[:N])/N)
plt.xlabel('Frequency ($Hz$)')
plt.ylabel('Amplitude ($Unit$)')

### The wrong Amplitude Spectrum because of Leakage Effect

The original signal.

In [ ]:
plt.plot(t,channel)
plt.xlabel('Time ($s$)')
plt.ylabel('Amplitude ($Unit$)')

In [ ]:
plt.plot(t, channel, label='Signal 1')
plt.plot(t+t[-1], channel, label='Signal 1 again')
plt.xlim(t[-1]-1, t[-1]+1)
plt.xlabel('Time ($s$)')
plt.ylabel('Amplitude ($Unit$)')
plt.legend()

In [ ]:
hamm = np.hamming(len(channel))
plt.plot(hamm)

In [ ]:

Yhamm = np.fft.fft(hamm*channel)

plt.figure(figsize=(7,3))
plt.subplot(121)
plt.plot(t,channel)
plt.title('Time Domain Signal')
plt.ylim(np.min(channel)*3, np.max(channel)*3)
plt.xlabel('Time ($s$)')
plt.ylabel('Amplitude ($Unit$)')

plt.subplot(122)
plt.plot(X, 2.0*np.abs(Yhamm[:N])/N)
plt.title('Frequency Domain Signal')
plt.xlabel('Frequency ($Hz$)')
plt.ylabel('Amplitude ($Unit$)')

plt.annotate("FFT",
            xy=(0.0, 0.1), xycoords='axes fraction',
            xytext=(-0.8, 0.2), textcoords='axes fraction',
            size=30, va="center", ha="center",
            arrowprops=dict(arrowstyle="simple",
                            connectionstyle="arc3,rad=0.2"))
plt.tight_layout()

plt.savefig('FFT.png',bbox_inches='tight', dpi=150, transparent=True)

In [ ]:
hann = np.hanning(len(channel))
plt.plot(hann)

In [ ]:
Y = np.fft.fft(hann*channel)
plt.plot(Y)

In [ ]:
N = round(len(Y)/2+1)
fa = 1.0/4 # every 15 minutes
print('fa=%.4fHz (Frequency)' % fa)

In [ ]:
X = np.linspace(0, fa/2, N, endpoint=True)

In [ ]:
FFTed_signal = 2.0*np.abs(Y[:N])/N
plt.plot(X, FFTed_signal)
plt.xlabel('Frequency ($Hz$)')
plt.ylabel('vertical power grid load ($MW$)')

In [ ]:
FFTed_signal

In [ ]:
times = np.linspace(0, 1, sampling_freq, endpoint=False)
sine = np.sin(20 * np.pi * times)
cosine = np.cos(10 * np.pi * times)
data = np.array([sine, cosine])

info = mne.create_info(ch_names=['10 Hz sine', '5 Hz cosine'],
                       ch_types=['misc'] * 2,
                       sfreq=sampling_freq)